# Mangaki Data Challenge

Featured on University of Big Data: https://bit.ly/mangakidatachallenge

## Load the data

In [1]:
import os, csv

DATA_DIR = '../data/mdc/'  # Here is the folder containing {train, test, watched}.csv

with open(os.path.join(DATA_DIR, 'watched.csv')) as f:
    next(f)
    triplets = [[int(user_id), int(work_id), rating] for user_id, work_id, rating in csv.reader(f)]

In [2]:
import pandas as pd

watched = pd.DataFrame(triplets, columns=['user_id', 'work_id', 'choice'])

Let's add a column that contains a numeric value for the rating. It will be used for matrix factorization (SVD).

In [31]:
rating_values = {'like': 2, 'love': 4, 'dislike': -2, 'neutral': 0.1}

watched['value'] = watched['choice'].map(rating_values)

In [32]:
watched.head()

,user_id,work_id,choice,value
0,717,8025,dislike,-2.0
1,1106,1027,neutral,0.1
2,1970,3949,neutral,0.1
3,1685,9815,like,2.0
4,1703,3482,like,2.0


In [33]:
import numpy as np

X = np.array(watched[['user_id', 'work_id']])
X.shape

(198970, 2)

We indeed have 198k rows and 2 columns (`user_id` and `work_id`).

In [34]:
nb_users = max(X[:, 0]) + 1  # Because users are numbered starting from 0
nb_works = max(X[:, 1]) + 1  # Because works are numbered starting from 0

In [35]:
y = np.array(watched['value'])

In [36]:
from mangaki.utils.svd import MangakiSVD

svd = MangakiSVD(20)
svd.set_parameters(nb_users, nb_works)
svd.fit(X, y)

Computing M: (1983 × 9897)


/Users/jin/Sites/mangaki/mangaki/mangaki/utils/svd.py:36: RuntimeWarning: invalid value encountered in double_scalars
  means[i] = np.sum(matrix[i]) / np.sum(matrix[i] != 0)


Chrono: fill and center matrix [0q, 647ms]
Shapes (1983, 20) (20,) (20, 9897)
Chrono: factor matrix [0q, 1796ms]


Now will user 1 like work 42? Let's figure out.

In [37]:
svd.predict(np.array([(1, 42)]))

array([ 2.28275862])

Apparently, yes. Did we predict correctly the training data?

In [39]:
svd.predict(X[:10])

array([-0.87987414,  0.08310813,  2.05496013,  2.07300611,  0.99431617,
        2.        ,  0.83510865,  2.        ,  2.41003277,  1.13191826])

In [41]:
y[:10]

array([-2. ,  0.1,  0.1,  2. ,  2. ,  2. , -2. ,  2. ,  2. ,  2. ])

Not so well. Let's try a different algorithm.

In [42]:
from mangaki.utils.als import MangakiALS

als = MangakiALS(20)
als.set_parameters(nb_users, nb_works)
als.fit(X, y)

Computing M: (1983 × 9897)
Chrono: fill and center matrix [0q, 887ms]
Shapes (1983, 20) (20, 9897)
Chrono: factor matrix [0q, 9472ms]


In [44]:
als.predict(X[:10])

array([ 0.63438461,  0.0385696 ,  1.5584425 ,  2.02189355,  0.87137735,
        2.        ,  0.48637334,  2.        ,  1.90309226,  1.7083744 ])

In [46]:
y[:10]

array([-2. ,  0.1,  0.1,  2. ,  2. ,  2. , -2. ,  2. ,  2. ,  2. ])

Let's compute the error of training.

In [49]:
y_pred_svd = svd.predict(X)
svd.compute_rmse(y_pred_svd, y)

1.1507216969289222

In [50]:
y_pred_als = als.predict(X)
svd.compute_rmse(y_pred_als, y)

0.87333921830246342

ALS fits the data better.

## Mais nous comme on est à Mangaki on a des objets plus simples

In [51]:
from mangaki.utils.data import Dataset

dataset = Dataset()
dataset.load_csv('ratings20170627.csv', convert=str, title_filename='works20170627.csv')

In [52]:
len(dataset.titles)

9897

In [57]:
dataset.anonymized

AnonymizedData(X=array([[717, 8025],
       [658, 8263],
       [1224, 1892],
       ..., 
       [1509, 3637],
       [299, 4184],
       [802, 8844]], dtype=object), y=array(['dislike', 'wontsee', 'willsee', ..., 'willsee', 'wontsee', 'like'], dtype=object), nb_users=1983, nb_works=9897)

Who has the most ratings in the dataset?

In [72]:
from collections import Counter

nb_ratings_of_user = Counter(dataset.anonymized.X[:, 0])
nb_ratings_of_user.most_common(5)

[(316, 3068), (985, 2483), (1940, 2179), (802, 2026), (1550, 1953)]

Which means user 316 has 3068 ratings!

In [67]:
import pandas as pd
import numpy as np

ratings = pd.DataFrame(np.column_stack([dataset.anonymized.X, dataset.anonymized.y]), columns=['user_id', 'work_id', 'choice'])

In [68]:
rating_values = {'love': 4, 'like': 2, 'dislike': -2, 'neutral': 0.1, 'willsee': 0.5, 'wontsee': -0.5}

In [69]:
ratings['value'] = ratings['choice'].map(rating_values)

In [71]:
ratings.head()

,user_id,work_id,choice,value
0,717,8025,dislike,-2.0
1,658,8263,wontsee,-0.5
2,1224,1892,willsee,0.5
3,1106,1027,neutral,0.1
4,1468,2743,wontsee,-0.5


## Computing the neighbors

In [73]:
from mangaki.utils.knn import MangakiKNN

knn = MangakiKNN(20)
knn.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
knn.fit(X, y)

In [87]:
knn.get_neighbors([316])

[array([ 716, 1263, 1683,   57, 1462,   19, 1272, 1655, 1433,  753,  179,
        1184,  767,  212, 1613,   39,  418, 1071,  856,  346])]

In [82]:
knn.predict(X[:10])

array([ 0.10828723,  0.        ,  0.        ,  2.17506037,  1.25183908,
        2.47958984,  1.28476562,  1.43611111,  1.88993939,  1.75133929])

In [84]:
y[:10]

array([-2. ,  0.1,  0.1,  2. ,  2. ,  2. , -2. ,  2. ,  2. ,  2. ])

## L'affaire KNN

Si je me connecte sur le site en tant qu'utilisateur qui a plus de 3000 ratings, voici ce qui se passe lorsque je cherche à obtenir des recommandations KNN.

    Chrono: get rated works [4q, 18ms]
    Chrono: make first anonymous data with 303214 ratings [5q, 3113ms]
    Chrono: prepare first fit [5q, 5286ms]
    Chrono: get neighbors [5q, 266ms]
    Chrono: fit knn-20 [6q, 221ms]
    Chrono: remove already rated [6q, 1ms]
    Chrono: compute every prediction [6q, 4ms]
    Chrono: get bulk [7q, 15ms]

Et ces valeurs sont évidemment plus longues sur la prod.

Plusieurs raisons sont en cause.

- Les algorithmes [`MangakiXXX`](https://github.com/mangaki/mangaki/tree/master/mangaki/mangaki/utils) (ex. `knn.py`, `als.py`) ont besoin de clés entre 0 et X-1. Or la prod a des trous dans les ID. C'est pour ça qu'on fait une conversion. Notez que les algos SVD et ALS vont directement puiser dans un backup dans le dossier `pickles`, c'est pour ça que ces algos sont plus rapides. Mais lorsqu'un nouvel utilisateur arrive sur le site, on n'a pas encore calculé sa SVD/ALS. Donc il faut recourir à KNN.
- Or, l'opération [`make_anonymous_data`](https://github.com/mangaki/mangaki/blob/master/mangaki/mangaki/utils/data.py#L89) est coûteuse. Elle fait deux boucles sur triplets, qui contient ici 300k éléments. En fait, réécrire la fonction `make_anonymous_data` en pandas serait plus rapide, j'ai l'impression.
- préparer first fit, c'est long aussi puisque ça parcourt les 300k ratings pour remplir la `lil_matrix` (cf. la fonction [`fit`](https://github.com/mangaki/mangaki/blob/master/mangaki/mangaki/utils/knn.py#L60) dans `knn.py`).

Une solution consisterait à stocker un objet qui a précalculé la matrice. Du coup, pour déterminer les plus proches voisins de quelqu'un on n'a plus qu'à faire une opération matricielle (cf. [`cosine_similarity`](https://github.com/mangaki/mangaki/blob/master/mangaki/mangaki/utils/knn.py#L36) dans `get_neighbors` de `knn.py`).

C'est votre mission.